In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
weights = ResNet18_Weights.IMAGENET1K_V1

train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=weights.transforms().mean,
        std=weights.transforms().std
    )
])

val_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=weights.transforms().mean,
        std=weights.transforms().std
    )
])

In [3]:
data_dir = "cnn_dataset"

train_ds = datasets.ImageFolder(f"{data_dir}/train", transform=train_tfms)
val_ds   = datasets.ImageFolder(f"{data_dir}/valid", transform=val_tfms)
test_ds  = datasets.ImageFolder(f"{data_dir}/test",  transform=val_tfms)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=128, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=128, shuffle=False)

class_names = train_ds.classes
num_classes = len(class_names)
print(class_names)

['10c', '10d', '10h', '10s', '2c', '2d', '2h', '2s', '3c', '3d', '3h', '3s', '4c', '4d', '4h', '4s', '5c', '5d', '5h', '5s', '6c', '6d', '6h', '6s', '7c', '7d', '7h', '7s', '8c', '8d', '8h', '8s', '9c', '9d', '9h', '9s', 'Ac', 'Ad', 'Ah', 'As', 'Jc', 'Jd', 'Jh', 'Js', 'Kc', 'Kd', 'Kh', 'Ks', 'Qc', 'Qd', 'Qh', 'Qs']


In [4]:
# set up the model
model = resnet18(weights=weights)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model.maxpool = nn.Identity()
model.fc = nn.Linear(model.fc.in_features, num_classes)

# freeze layers
for param in model.parameters():
    param.requires_grad = False
# unfreeze last layer
for param in model.fc.parameters(): 
    param.requires_grad = True

In [5]:
def get_free_gpu():
    if not torch.cuda.is_available():
        return torch.device("cpu")

    free_mem = []
    for i in range(torch.cuda.device_count()):
        torch.cuda.set_device(i)
        torch.cuda.empty_cache()
        stats = torch.cuda.mem_get_info(i)
        free_mem.append((stats[0], i))  

    _, best_gpu = max(free_mem)
    return torch.device(f"cuda:{best_gpu}")

device = get_free_gpu()
model.to(device)
print("Using device:", device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-3
)

Using device: cuda:6


In [6]:
def train_one_epoch(model, loader, epoch=None):
    model.train()
    running_loss, correct, total = 0, 0, 0

    pbar = tqdm(loader, desc=f"Epoch {epoch} [train]", leave=False)

    for x, y in pbar:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        preds = out.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)

        pbar.set_postfix(
            loss=running_loss / total,
            acc=correct / total
        )

    return running_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0

    for x, y in tqdm(loader, desc="Validation", leave=False):
        x, y = x.to(device), y.to(device)
        out = model(x)
        preds = out.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    return correct / total


In [7]:
# --- Phase 1: Train head only (frozen backbone) ---
for epoch in range(7):  # train 7 epochs first
    train_loss, train_acc = train_one_epoch(
        model, train_loader, epoch=epoch+1
    )
    val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}: "
          f"loss={train_loss:.4f}, "
          f"train_acc={train_acc:.3f}, "
          f"val_acc={val_acc:.3f}")

# --- Phase 2: Unfreeze layer4 for fine-tuning ---
for param in model.layer4.parameters():
    param.requires_grad = True

optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4  # lower learning rate for fine-tuning
)

for epoch in range(7, 14):  # fine-tune for 7 more epochs
    train_loss, train_acc = train_one_epoch(model, train_loader, epoch=epoch+1)
    val_acc = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}: "
          f"loss={train_loss:.4f}, "
          f"train_acc={train_acc:.3f}, "
          f"val_acc={val_acc:.3f}")

Epoch 1: loss=3.4679, train_acc=0.122, val_acc=0.170


Epoch 2: loss=3.0318, train_acc=0.201, val_acc=0.224


Epoch 3: loss=2.8485, train_acc=0.235, val_acc=0.248


Epoch 4: loss=2.7252, train_acc=0.259, val_acc=0.258


Epoch 5: loss=2.6422, train_acc=0.274, val_acc=0.282


...

Epoch 10: loss=0.3759, train_acc=0.923, val_acc=0.920


Epoch 11: loss=0.2422, train_acc=0.958, val_acc=0.958


Epoch 12: loss=0.1613, train_acc=0.977, val_acc=0.965


Epoch 13: loss=0.1177, train_acc=0.984, val_acc=0.974


Epoch 14: loss=0.0813, train_acc=0.991, val_acc=0.979


In [8]:
def evaluate_test(model, loader):
    model.eval()
    correct = 0
    total = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    return total_loss / len(loader), correct / total


# Run test
test_loss, test_acc = evaluate_test(model, test_loader)

print(f"\nFinal Test Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.3f}")


Final Test Results:
Test Loss: 0.1041
Test Accuracy: 0.978


In [9]:
torch.save(model.state_dict(), "cnn_resnet18_224x224.pt")
